In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s6e1/sample_submission.csv
/kaggle/input/playground-series-s6e1/train.csv
/kaggle/input/playground-series-s6e1/test.csv


In [2]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor

Load Data

In [3]:
import os

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/playground-series-s6e1/sample_submission.csv
/kaggle/input/playground-series-s6e1/train.csv
/kaggle/input/playground-series-s6e1/test.csv


In [4]:
import pandas as pd

train_df = pd.read_csv("/kaggle/input/playground-series-s6e1/train.csv")
test_df  = pd.read_csv("/kaggle/input/playground-series-s6e1/test.csv")

print(train_df.head())
print(train_df.columns)

   id  age  gender   course  study_hours  class_attendance internet_access  \
0   0   21  female     b.sc         7.91              98.8              no   
1   1   18   other  diploma         4.95              94.8             yes   
2   2   20  female     b.sc         4.68              92.6             yes   
3   3   19    male     b.sc         2.00              49.5             yes   
4   4   23    male      bca         7.65              86.9             yes   

   sleep_hours sleep_quality   study_method facility_rating exam_difficulty  \
0          4.9       average  online videos             low            easy   
1          4.7          poor     self-study          medium        moderate   
2          5.8          poor       coaching            high        moderate   
3          8.3       average    group study            high        moderate   
4          9.6          good     self-study            high            easy   

   exam_score  
0        78.3  
1        46.7  
2       

In [5]:
import pandas as pd

print("START")

# Load the CSV
train_df = pd.read_csv("/kaggle/input/playground-series-s6e1/train.csv")
print("TRAIN DF LOADED")

# Inspect columns carefully
print("Columns in CSV:")
print(train_df.columns.tolist())

# Strip any leading/trailing spaces from column names
train_df.columns = train_df.columns.str.strip()

# Check again after stripping
print("Columns after stripping spaces:")
print(train_df.columns.tolist())

# Make sure 'math_score' exists
if "math_score" in train_df.columns:
    X = train_df.drop("math_score", axis=1)
    y = train_df["math_score"]
    print("X and y created")
    print(X.shape)
else:
    print("Column 'math_score' not found. Check column names above.")

print("END")

START
TRAIN DF LOADED
Columns in CSV:
['id', 'age', 'gender', 'course', 'study_hours', 'class_attendance', 'internet_access', 'sleep_hours', 'sleep_quality', 'study_method', 'facility_rating', 'exam_difficulty', 'exam_score']
Columns after stripping spaces:
['id', 'age', 'gender', 'course', 'study_hours', 'class_attendance', 'internet_access', 'sleep_hours', 'sleep_quality', 'study_method', 'facility_rating', 'exam_difficulty', 'exam_score']
Column 'math_score' not found. Check column names above.
END


In [6]:
import pandas as pd

# Load train CSV
train_df = pd.read_csv("/kaggle/input/playground-series-s6e1/train.csv")

# Show columns exactly as they are
print("Columns in train_df:")
for col in train_df.columns:
    print(f"'{col}'")


Columns in train_df:
'id'
'age'
'gender'
'course'
'study_hours'
'class_attendance'
'internet_access'
'sleep_hours'
'sleep_quality'
'study_method'
'facility_rating'
'exam_difficulty'
'exam_score'


In [7]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# 1. LOAD DATA
train_df = pd.read_csv("/kaggle/input/playground-series-s6e1/train.csv")
test_df = pd.read_csv("/kaggle/input/playground-series-s6e1/test.csv")

# 2. CLEAN COLUMN NAMES
# This handles the KeyError by ensuring names are stripped and lowercase
train_df.columns = train_df.columns.str.strip().str.lower()
test_df.columns = test_df.columns.str.strip().str.lower()

# Dynamically find the target column (it's 'exam_score' in this dataset)
target_col = [col for col in train_df.columns if 'score' in col][0]
print(f"Target column identified: {target_col}")

# 3. PREPARE X AND Y
X = train_df.drop(columns=['id', target_col])
y = train_df[target_col]
X_test = test_df.drop(columns=['id'])

# 4. ENCODE CATEGORICAL DATA
# We use a loop to handle unseen labels in the test set
cat_cols = X.select_dtypes(include=['object']).columns

for col in cat_cols:
    le = LabelEncoder()
    # Fit on both train and test to avoid "unseen label" errors
    full_labels = pd.concat([X[col], X_test[col]], axis=0).astype(str)
    le.fit(full_labels)
    X[col] = le.transform(X[col].astype(str))
    X_test[col] = le.transform(X_test[col].astype(str))

# 5. TRAIN/TEST SPLIT (Validation)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# 6. TRAIN THE MODEL (This fixes your NameError)
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# 7. EVALUATE
val_preds = model.predict(X_val)
rmse = np.sqrt(mean_squared_error(y_val, val_preds))
print(f"Validation RMSE: {rmse:.4f}")

# 8. CREATE FINAL SUBMISSION
test_preds = model.predict(X_test)
submission = pd.DataFrame({
    "id": test_df["id"],
    "exam_score": test_preds
})

Target column identified: exam_score
Validation RMSE: 9.1213


In [8]:
# 8. CREATE FINAL SUBMISSION
# Use the actual 'id' column from the test_df, not the index
# Use the exact column names expected by the competition
submission = pd.DataFrame({
    "id": test_df["id"],
    "exam_score": test_preds
})

# Save to CSV
submission.to_csv("submission.csv", index=False)
print("submission.csv created successfully with correct column names!")

submission.csv created successfully with correct column names!
